[View in Colaboratory](https://colab.research.google.com/github/ami432/INKERS/blob/master/Bharatkumar_Batch3_Assignment4B-87_8.ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size =64
num_classes = 10
epochs = 30
l = 9
num_filter = 70
compression = 0.5
dropout_rate = 0.3

In [6]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 29s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 14, dropout_rate = 0.3):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 14, dropout_rate = 0.3):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 70
dropout_rate = 0.3
l = 9
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [11]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 70)   1890        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 70)   280         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 70)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
# checkpoint
filepath="CIFAR10-weights-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode='auto',period=2)
callbacks_list = [checkpoint]


In [14]:
# Fit the model
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,callbacks=callbacks_list,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
43776/50000 [=========================>....] - ETA: 56s - loss: 1.4637 - acc: 0.4628

50000/50000 [==============================] - 483s 10ms/step - loss: 1.4211 - acc: 0.4797 - val_loss: 1.8448 - val_acc: 0.4766
Epoch 2/30
26112/50000 [==============>...............] - ETA: 3:32 - loss: 1.0169 - acc: 0.6360

50000/50000 [==============================] - 472s 9ms/step - loss: 0.9636 - acc: 0.6555 - val_loss: 2.2382 - val_acc: 0.4910

Epoch 00002: saving model to CIFAR10-weights-02-0.49.hdf5
Epoch 3/30
15360/50000 [========>.....................] - ETA: 5:08 - loss: 0.8256 - acc: 0.7089

50000/50000 [==============================] - 472s 9ms/step - loss: 0.7848 - acc: 0.7235 - val_loss: 1.0314 - val_acc: 0.6774
Epoch 4/30
15424/50000 [========>.....................] - ETA: 5:07 - loss: 0.7028 - acc: 0.7461

50000/50000 [==============================] - 472s 9ms/step - loss: 0.6766 - acc: 0.7609 - val_loss: 1.2496 - val_acc: 0.6521

Epoch 00004: saving model to CIFAR10-weights-04-0.65.hdf5
Epoch 5/30
12160/50000 [======>.......................] - ETA: 5:36 - loss: 0.6114 - acc: 0.7838

...

50000/50000 [==============================] - 467s 9ms/step - loss: 0.1768 - acc: 0.9370 - val_loss: 0.5260 - val_acc: 0.8669

Epoch 00026: saving model to CIFAR10-weights-26-0.87.hdf5
Epoch 27/30
11648/50000 [=====>........................] - ETA: 5:37 - loss: 0.1575 - acc: 0.9412

50000/50000 [==============================] - 467s 9ms/step - loss: 0.1676 - acc: 0.9384 - val_loss: 0.7307 - val_acc: 0.8403
Epoch 28/30
13952/50000 [=======>......................] - ETA: 5:17 - loss: 0.1554 - acc: 0.9455

50000/50000 [==============================] - 468s 9ms/step - loss: 0.1624 - acc: 0.9423 - val_loss: 0.6783 - val_acc: 0.8516

Epoch 00028: saving model to CIFAR10-weights-28-0.85.hdf5
Epoch 29/30
11648/50000 [=====>........................] - ETA: 5:36 - loss: 0.1458 - acc: 0.9485

50000/50000 [==============================] - 467s 9ms/step - loss: 0.1565 - acc: 0.9450 - val_loss: 0.5222 - val_acc: 0.8706
Epoch 30/30
13952/50000 [=======>......................] - ETA: 5:15 - loss: 0.1447 - acc: 0.9486

50000/50000 [==============================] - 465s 9ms/step - loss: 0.1520 - acc: 0.9451 - val_loss: 0.5351 - val_acc: 0.8734

Epoch 00030: saving model to CIFAR10-weights-30-0.87.hdf5


In [15]:
# Model is still stabilising/settling
#Save the trained weights after 30 Epoch's into .h5 format
model.save_weights("CIFAR-10_model_weights_after30_epochs.h5")
print("Saved model to disk")

from google.colab import files

files.download('CIFAR-10_model_weights_after30_epochs.h5')

Saved model to disk


In [0]:
model.load_weights('CIFAR-10_model_weights_after30_epochs.h5')

In [19]:
# Running the Model again for  10 more Epoch's.So previous 30 + current 10 = 40 Epoch's
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,callbacks=callbacks_list,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
43776/50000 [=========================>....] - ETA: 54s - loss: 0.1433 - acc: 0.9487

50000/50000 [==============================] - 468s 9ms/step - loss: 0.1447 - acc: 0.9484 - val_loss: 0.5786 - val_acc: 0.8636
Epoch 2/10
26240/50000 [==============>...............] - ETA: 3:29 - loss: 0.1353 - acc: 0.9514

50000/50000 [==============================] - 468s 9ms/step - loss: 0.1416 - acc: 0.9489 - val_loss: 0.6440 - val_acc: 0.8560

Epoch 00002: saving model to CIFAR10-weights-02-0.86.hdf5
Epoch 3/10
15424/50000 [========>.....................] - ETA: 5:04 - loss: 0.1278 - acc: 0.9537

50000/50000 [==============================] - 468s 9ms/step - loss: 0.1343 - acc: 0.9523 - val_loss: 0.6128 - val_acc: 0.8641
Epoch 4/10
15424/50000 [========>.....................] - ETA: 5:04 - loss: 0.1211 - acc: 0.9560

50000/50000 [==============================] - 468s 9ms/step - loss: 0.1365 - acc: 0.9513 - val_loss: 0.5957 - val_acc: 0.8675

Epoch 00004: saving model to CIFAR10-weights-04-0.87.hdf5
Epoch 5/10
12160/50000 [======>.......................] - ETA: 5:33 - loss: 0.1197 - acc: 0.9593

...

50000/50000 [==============================] - 468s 9ms/step - loss: 0.1223 - acc: 0.9556 - val_loss: 0.5788 - val_acc: 0.8600

Epoch 00006: saving model to CIFAR10-weights-06-0.86.hdf5
Epoch 7/10
11776/50000 [======>.......................] - ETA: 5:37 - loss: 0.1097 - acc: 0.9608

50000/50000 [==============================] - 469s 9ms/step - loss: 0.1189 - acc: 0.9574 - val_loss: 0.5408 - val_acc: 0.8759
Epoch 8/10
14016/50000 [=======>......................] - ETA: 5:17 - loss: 0.1112 - acc: 0.9603

50000/50000 [==============================] - 469s 9ms/step - loss: 0.1163 - acc: 0.9587 - val_loss: 0.5159 - val_acc: 0.8841

Epoch 00008: saving model to CIFAR10-weights-08-0.88.hdf5
Epoch 9/10
11712/50000 [======>.......................] - ETA: 5:37 - loss: 0.1022 - acc: 0.9635

50000/50000 [==============================] - 468s 9ms/step - loss: 0.1136 - acc: 0.9595 - val_loss: 0.6396 - val_acc: 0.8713
Epoch 10/10
13952/50000 [=======>......................] - ETA: 5:17 - loss: 0.1016 - acc: 0.9637

50000/50000 [==============================] - 468s 9ms/step - loss: 0.1079 - acc: 0.9613 - val_loss: 0.5490 - val_acc: 0.8789

Epoch 00010: saving model to CIFAR10-weights-10-0.88.hdf5


In [20]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 33s 3ms/step
Test loss: 0.5490125443607569
Test accuracy: 0.8789


In [21]:
# Save the trained weights in to .h5 format
model.save_weights("CIFAR-10_model_aacuracy.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('CIFAR-10_model_aacuracy.h5')